In [2]:
import pandas as pd
import pandas as pd
import esm
import numpy as np
import torch
from tqdm import tqdm

df = pd.read_csv('/home/musong/Desktop/paper/data/drugfinder/esm2_320_dimensions_with_labels.csv')

df.columns

Index(['UniProt_id', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '312', '313', '314', '315', '316', '317', '318', '319', '320', 'label'],
      dtype='object', length=322)

In [3]:
def esm_embeddings(peptide_sequence_list):
    model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    model.eval()  # disables dropout for deterministic results
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    ## batch tokens are the embedding results of the whole data set
    batch_tokens = batch_tokens.to(device)
    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        # Here we export the last layer of the EMS model output as the representation of the peptides
        # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append((peptide_sequence_list[i][0], token_representations[i, 1 : tokens_len - 1].mean(0)))
    return sequence_representations

In [4]:
a = 'Q9H2Y9'
b = """
MDEGTGLQPGAGEQLEAPATAEAVQERCEPETLRSKSLPVLSSASCRPSLSPTSGDANPAFGCVDSSGHQELKQGPNPLAPSPSAPSTSAGLGDCNHRVDLSKTFSVSSALAMLQERRCLYVVLTDSRCFLVCMCFLTFIQALMVSGYLSSVITTIERRYSLKSSESGLLVSCFDIGNLVVVVFVSYFGGRGRRPLWLAVGGLLIAFGAALFALPHFISPPYQIQELNASAPNDGLCQGGNSTATLEPPACPKDSGGNNHWVYVALFICAQILIGMGSTPIYTLGPTYLDDNVKKENSSLYLAIMYVMGALGPAVGYLLGGLLIGFYVDPRNPVHLDQNDPRFIGNWWSGFLLCAIAMFLVIFPMFTFPKKLPPRHKKKKKKKFSVDAVSDDDVLKEKSNNSEQADKKVSSMGFGKDVRDLPRAAVRILSNMTFLFVSLSYTAESAIVTAFITFIPKFIESQFGIPASNASIYTGVIIVPSAGVGIVLGGYIIKKLKLGARESAKLAMICSGVSLLCFSTLFIVGCESINLGGINIPYTTGPSLTMPHRNLTGSCNVNCGCKIHEYEPVCGSDGITYFNPCLAGCVNSGNLSTGIRNYTECTCVQSRQVITPPTVGQRSQLRVVIVKTYLNENGYAVSGKCKRTCNTLIPFLVFLFIVTFITACAQPSAIIVTLRSVEDEERPFALGMQFVLLRTLAYIPTPIYFGAVIDTTCMLWQQECGVQGSCWEYNVTSFRFVYFGLAAGLKFVGFIFIFLAWYSIKYKEDGLQRRRQREFPLSTVSERVGHPDNARTRSCPAFSTQGEFHEETGLQKGIQCAAQTYPGPFPEAISSSADPGLEESPAALEPPS
"""
esm_embeddings([(a, b)])

[('Q9H2Y9',
  tensor([-2.9326e-02, -7.5480e-02,  1.5323e-01,  2.2830e-01,  2.7175e-02,
           1.4604e-02, -2.1099e-02,  4.4349e-02, -1.1190e-02, -1.4053e-01,
           1.7994e-01,  2.0112e-01, -5.4789e-02,  8.6889e-02,  1.1479e-01,
          -1.4297e-01, -9.6325e-02,  5.1462e-02,  2.3146e-02,  8.7105e-02,
          -1.0492e-01, -7.3735e-02,  6.3370e-02,  8.6488e-02,  6.0477e-03,
           2.7618e-01,  3.6619e-02, -1.9149e-03,  1.1605e-01,  2.3564e-01,
          -5.1230e-02,  1.1305e-02,  1.6400e-01, -2.3677e-01,  1.5775e-01,
          -2.7568e-01,  1.6861e-01,  1.2491e-01,  5.2414e-02,  1.1218e-01,
          -8.4583e-02,  2.8117e-02,  2.4195e-01,  9.7498e-02, -8.7013e-02,
           1.8309e-02, -8.8874e-01,  1.8250e-01,  1.1422e-02, -1.0467e-01,
           1.5478e-02,  5.6556e-02, -8.5190e-02,  1.5708e-01, -2.1839e-01,
          -2.7518e-01,  1.5420e-01, -9.3446e-02,  8.1160e-02, -2.8677e-01,
           1.6532e-02, -5.2077e-03, -4.0684e+00, -2.5211e-01,  1.1528e-01,
          -1.